# Octonion Multiplication

Examples from blog posts by [John D. Cook](https://www.johndcook.com/blog/services-2/)

## How far is xy from yx on average for quaternions? [5 July 2018](https://www.johndcook.com/blog/2018/07/05/quaternion-multiplication/)

In [7]:
import numpy as np
    
def random_unit_quaternion():
    x = np.random.normal(size=4)
    return x / np.linalg.norm(x)
    
def mult(x, y):
    return np.array([
        x[0]*y[0] - x[1]*y[1] - x[2]*y[2] - x[3]*y[3],
        x[0]*y[1] + x[1]*y[0] + x[2]*y[3] - x[3]*y[2],
        x[0]*y[2] - x[1]*y[3] + x[2]*y[0] + x[3]*y[1],
        x[0]*y[3] + x[1]*y[2] - x[2]*y[1] + x[3]*y[0]
    ])
    
    
N = 10000
s = 0
for _ in range(N):
    x = random_unit_quaternion()
    y = random_unit_quaternion()
    s += np.linalg.norm(mult(x, y) - mult(y, x))
  
print(s/N)

1.129737502391203


## Weakening the requirements of a group [8 July 2018](https://www.johndcook.com/blog/2018/07/08/weak-groups/)

## Python code for octonion and sedenion multiplication [9 July 2018](https://www.johndcook.com/blog/2018/07/09/octonioin-multiplication/)

In [8]:
import numpy as np

# quaternion multiplication
def qmult(x, y):
    return np.array([
        x[0]*y[0] - x[1]*y[1] - x[2]*y[2] - x[3]*y[3],
        x[0]*y[1] + x[1]*y[0] + x[2]*y[3] - x[3]*y[2],
        x[0]*y[2] - x[1]*y[3] + x[2]*y[0] + x[3]*y[1],
        x[0]*y[3] + x[1]*y[2] - x[2]*y[1] + x[3]*y[0]
   ])

# quaternion conjugate
def qstar(x):
    return x*np.array([1, -1, -1, -1])

# octonion multiplication
def omult(x, y):
    # Split octonions into pairs of quaternions
    a, b = x[:4], x[4:]
    c, d = y[:4], y[4:]
   
    z = np.zeros(8)
    z[:4] = qmult(a, c) - qmult(qstar(d), b)
    z[4:] = qmult(d, a) + qmult(b, qstar(c))
    return z

In [9]:
from numpy.linalg import norm

def random_unit_octonion():
    x = np.random.normal(size=8)
    return x / norm(x)

x = random_unit_octonion()
y = random_unit_octonion()
z = random_unit_octonion()

In [17]:
print(x)
print(y)
print(z)

[ 0.73496342 -0.07692169 -0.51074567  0.25060754  0.07788417  0.14005789
  0.01241653 -0.3231257 ]
[-0.45353352  0.44214854  0.14672527  0.39704963 -0.16636928 -0.18117144
 -0.46888457  0.37320335]
[-0.71368288 -0.21918625 -0.00957934  0.48588364 -0.11007475  0.1893148
  0.2200518   0.33175311]


In [18]:
eps = 1e-15

# alternative identities

a = omult(omult(x, x), y)
b = omult(x, omult(x, y))
assert( norm(a - b) < eps )

a = omult(omult(x, y), y)
b = omult(x, omult(y, y))
assert( norm(a - b) < eps )

In [19]:
# Moufang identities
    
a = omult(z, omult(x, omult(z, y)))
b = omult(omult(omult(z, x), z), y)
assert( norm(a - b) < eps )

a = omult(x, omult(z, omult(y, z)))
b = omult(omult(omult(x, z), y), z)
assert( norm(a - b) < eps )

a = omult(omult(z,x), omult(y, z))
b = omult(omult(z, omult(x, y)), z)
assert( norm(a  - b) < eps )

a = omult(omult(z,x), omult(y, z))
b = omult(z, omult(omult(x, y), z))
assert( norm(a - b) < eps )

In [20]:
# norm condition
n = norm(omult(omult(x, y), z))
assert( abs(n - 1) < eps )

In [21]:
def ostar(x):
    mask = -np.ones(8)
    mask[0] = 1
    return x*mask
 
# sedenion multiplication
def smult(x, y):
    # Split sedenions into pairs of octonions
    a, b = x[:8], x[8:]
    c, d = y[:8], y[8:]
    z = np.zeros(16)
    z[:8] = omult(a, c) - omult(ostar(d), b)
    e[8:] = omult(d, a) + omult(b, ostar(c))
    return z

In [22]:
x

array([ 0.73496342, -0.07692169, -0.51074567,  0.25060754,  0.07788417,
        0.14005789,  0.01241653, -0.3231257 ])

In [23]:
ostar(x)

array([ 0.73496342,  0.07692169,  0.51074567, -0.25060754, -0.07788417,
       -0.14005789, -0.01241653,  0.3231257 ])

## Refactored code for division algebras [10 July 2018](https://www.johndcook.com/blog/2018/07/10/cayley-dickson/)

In [24]:
def conj(x):
        xstar = -x
        xstar[0] *= -1
        return xstar 

def CayleyDickson(x, y):
    n = len(x)

    if n == 1:
        return x*y

    m = n // 2

    a, b = x[:m], x[m:]
    c, d = y[:m], y[m:]
    z = np.zeros(n)
    z[:m] = CayleyDickson(a, c) - CayleyDickson(conj(d), b)
    z[m:] = CayleyDickson(d, a) + CayleyDickson(b, conj(c))
    return z

In [26]:
print(x)
print(conj(x))

[ 0.73496342 -0.07692169 -0.51074567  0.25060754  0.07788417  0.14005789
  0.01241653 -0.3231257 ]
[ 0.73496342  0.07692169  0.51074567 -0.25060754 -0.07788417 -0.14005789
 -0.01241653  0.3231257 ]


In [28]:
print(CayleyDickson(x, y))

[-0.15913853  0.27635344  0.44010145  0.43142703 -0.5690915  -0.19754899
 -0.06152452  0.38962692]
